In [ ]:
Face Recognition in Deep Learning

In [ ]:
In this Project used models - 
1) DeepFace
2) FaceNet
3) VGGNet
4) FaceRecognition Library

In [ ]:
FaceRecognition library gives us higher Accuracy because no need of any models.
Then Facenet and DeepFace almost gives similar Accuracy.
VGGNet gives low Accuracy.

In [ ]:
Install following Packages -
Numpy
Keras
deepface
PIL
tkinter
cv2
os
tensorflow
mtcnn

In [ ]:
install the following and store in same working folder
1) dlib-19.24.1-cp311-cp311-win_amd64.whl
2) facenet_keras.h5
3) vgg_face_weights.h5

In [ ]:
Create two folders namely imagesatted and imagebasics

Store some images in imagesatted for recognition and dataset purpose.

In [ ]:
Create Attendence.csv file

In [ ]:
Create the following python files.and also while running each separate file change their names into gui2.py and GUI.py.
for ex. if you want to run DeepFace.py file then instead of AttendenceProject.py file change its name to DeepFace.py file into gui2.py file and GUI.py

In [1]:
#GUI.py

import tkinter as tk
import subprocess


def mark_attendance():
    # Call the attendance.py file using subprocess
    subprocess.Popen(["python", "DeepFace.py"])

# Create the main application window
root = tk.Tk()
root.title("Attendance System")

# Create a button for marking attendance
mark_button = tk.Button(root, text="Mark Attendance", command=mark_attendance)
mark_button.pack(pady=20)

# Run the Tkinter event loop
root.mainloop()


In [ ]:
#AttendenceProject.py - FaceRecognition Library.

import face_recognition
import cv2
import numpy as np
import os
from datetime import datetime
import tkinter as tk
from PIL import ImageTk, Image

def button():
    path= 'imagesatted'
    images=[]
    classNames=[]
    mylist=os.listdir(path)
    print(mylist)
    for cls in mylist:
        curImg=cv2.imread(f'{path}/{cls}')
        images.append(curImg)
        classNames.append(os.path.splitext(cls)[0])
    print(classNames)
    def findEncodings(images):
        encodeList =[]
        for img in images:
            img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            encode=face_recognition.face_encodings(img)[0]
            encodeList.append(encode)
        return encodeList
    def markAttendence(name):
        with open('Attendance.csv', 'r+') as f:
            myDataList = f.readlines()
            print(myDataList)
            nameList=[]
            for line in myDataList:
                entry= line.split(',')
                nameList.append(entry[0])
            if name not in nameList:
                now = datetime.now()
                dtString=now.strftime('%H:%M:%S')
                f.writelines(f'\n{name},{dtString}')

    encodeListKnown = findEncodings(images)
    print('Encoding complete')
    cap=cv2.VideoCapture(0)
    total_attempts = 0
    correct_recognitions = 0
    while True:
        success, img=cap.read()
        imgS = cv2.resize(img,(0,0),None,0.25,0.25)
        imgS=cv2.cvtColor(imgS,cv2.COLOR_BGR2RGB)
        facesCurFrame =face_recognition.face_locations(imgS)#TOP < BOTTOM< LEFT < RIGHT
        encodesCurFrame=face_recognition.face_encodings(imgS,facesCurFrame)
        #print(encodesCurFrame)
        for encodeFace,faceLoc in zip(encodesCurFrame,facesCurFrame):
            #print(encodeFace)
            total_attempts += 1
            print(faceLoc)
            matches= face_recognition.compare_faces(encodeListKnown,encodeFace)
            print(matches)
            faceDis=face_recognition.face_distance(encodeListKnown,encodeFace)
            #print(faceDis)
            matchIndex=np.argmin(faceDis)
            if matches[matchIndex]:
                correct_recognitions += 1
                name= classNames[matchIndex].upper()
                accuracy = (correct_recognitions / total_attempts) * 100  # Calculate accuracy

                y1,x2,y2,x1=faceLoc
                y1,x2,y2,x1= y1*4,x2*4,y2*4,x1*4

                print(name)
                cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
                cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
                cv2.putText(img,f'{name} ({accuracy:.2f}%)',(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2) #2- thickness
                markAttendence(name)
                print("Attendance marked for", name)


        #shows live camera image
        cv2.imshow('Webcam',img)
        cv2.waitKey(1)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Create the main application window
root = tk.Tk()
root.title("Attendance System")

# Create a button for marking attendance
def hide_window():
 root.destroy()
root.protocol("WM_DELETE_WINDOW", hide_window) # Hide window when close button is clicked
root.maxsize(width=800, height=600)
root.title("Face Recognition System")
root.geometry("800x600")
image = Image.open("imagebasic/bg.jpg") # Replace "path_to_image_file.jpg" with the actual path to your
#image file
image = image.resize((root.winfo_screenwidth(), root.winfo_screenheight()),)
background_image = ImageTk.PhotoImage(image)
background_label = tk.Label(root, image=background_image)
background_label.place(x=0, y=0, relwidth=1, relheight=1)
l = tk.Label(root, text='Welcome Face Recognition System', font="Calibri 18 bold", bg='#CAEBF0')
l.place(x=250, y=100)
mark_button = tk.Button(root, text="Mark Attendance", command=button)
mark_button.pack(pady=20)
mark_button.place(x=350,y=250)

# Run the Tkinter event loop
root.mainloop()



In [ ]:
#DeepFace.py

import cv2
import numpy as np
import os
from datetime import datetime
from deepface import DeepFace

# Path to your images folder
path = 'imagesatted'
images = []
classNames = []

# List the files in the images folder
mylist = os.listdir(path)
print(mylist)

# Load images and extract class names
for cls in mylist:
    curImg = cv2.imread(os.path.join(path, cls))
    if curImg is not None:
        images.append(curImg)
        classNames.append(os.path.splitext(cls)[0])
print(classNames)

# Function to find face encodings using DeepFace
def findEncodings(images):
    encodeList = []
    for img in images:
        try:
            encodings = DeepFace.represent(img, model_name='VGG-Face', enforce_detection=False)
            if encodings:
                encodeList.append(encodings[0]["embedding"])
        except Exception as e:
            print(f"Error encoding image: {e}")
    return encodeList

# Function to mark attendance
def markAttendance(name):
    with open('Attendance.csv', 'r+') as f:
        myDataList = f.readlines()
        nameList = [line.split(',')[0] for line in myDataList]
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')

# Find encodings for known faces
encodeListKnown = findEncodings(images)
print('Encoding complete')

# Initialize webcam
cap = cv2.VideoCapture(0)

total_attempts = 0
correct_recognitions = 0

while True:
    success, img = cap.read()
    if not success:
        break
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    try:
        # Detect faces in the current frame
        facesCurFrame = DeepFace.extract_faces(imgS, detector_backend='mtcnn', enforce_detection=False)
    except Exception as e:
        print(f"Detection error: {e}")
        continue

    for face in facesCurFrame:
        faceLoc = face["facial_area"]
        faceImg = face["face"]
        try:
            encodeFace = DeepFace.represent(faceImg, model_name='VGG-Face', enforce_detection=False)[0]["embedding"]
        except Exception as e:
            print(f"Representation error: {e}")
            continue

        distances = np.linalg.norm(np.array(encodeListKnown) - encodeFace, axis=1)
        matchIndex = np.argmin(distances)
        total_attempts += 1  # Increment total attempts
        if distances[matchIndex] < 0.6:  # Threshold for a valid recognition
            correct_recognitions += 1  # Increment correct recognitions
            name = classNames[matchIndex].upper()
            y1, x2, y2, x1 = faceLoc["y"], faceLoc["x"] + faceLoc["w"], faceLoc["y"] + faceLoc["h"], faceLoc["x"]
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            markAttendance(name)
            print("Attendance marked for", name)
        else:
            print(f"No match found. Distances: {distances}")

    # Show live camera image
    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Calculate and print accuracy
if total_attempts > 0:
    accuracy = (correct_recognitions / total_attempts) * 100
    print(f'Accuracy: {accuracy:.2f}%')
else:
    print('No face recognition attempts made.')


In [ ]:
#FAceNet.py

import numpy as np
import cv2
import os
from datetime import datetime
from tensorflow.keras.models import load_model
from mtcnn import MTCNN
from numpy import asarray, expand_dims

# Load the pre-trained FaceNet model
model = load_model('facenet_keras.h5')

# Function to preprocess the face for FaceNet model
def preprocess_face(face):
    face = cv2.resize(face, (160, 160))
    face = face.astype('float32')
    mean, std = face.mean(), face.std()
    face = (face - mean) / std
    face = expand_dims(face, axis=0)
    return face

# Function to get the face embedding using FaceNet
def get_embedding(model, face):
    face = preprocess_face(face)
    embedding = model.predict(face)
    return embedding[0]

# Path to your images folder
path = 'imagesatted'
images = []
classNames = []
mylist = os.listdir(path)
print(mylist)
for cls in mylist:
    curImg = cv2.imread(os.path.join(path, cls))
    if curImg is not None:
        images.append(curImg)
        classNames.append(os.path.splitext(cls)[0])
print(classNames)

# Initialize MTCNN face detector
detector = MTCNN()

# Function to find face encodings for all images in the folder
def findEncodings(images):
    encodeList = []
    for img in images:
        results = detector.detect_faces(img)
        if results:
            x1, y1, width, height = results[0]['box']
            x2, y2 = x1 + width, y1 + height
            face = img[y1:y2, x1:x2]
            encode = get_embedding(model, face)
            encodeList.append(encode)
    return encodeList

# Function to mark attendance
def markAttendance(name):
    with open('Attendance.csv', 'r+') as f:
        myDataList = f.readlines()
        nameList = [line.split(',')[0] for line in myDataList]
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')

# Find encodings for known faces
encodeListKnown = findEncodings(images)
print('Encoding complete')

# Initialize webcam
cap = cv2.VideoCapture(0)

total_attempts = 0
correct_recognitions = 0

while True:
    success, img = cap.read()
    if not success or img is None:
        print("Failed to capture image from camera.")
        continue

    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    facesCurFrame = detector.detect_faces(imgS)
    encodesCurFrame = []

    for result in facesCurFrame:
        x1, y1, width, height = result['box']
        x2, y2 = x1 + width, y1 + height
        face = imgS[y1:y2, x1:x2]
        encode = get_embedding(model, face)
        encodesCurFrame.append((encode, (x1, y1, x2, y2)))

    for encodeFace, faceLoc in encodesCurFrame:
        total_attempts += 1
        distances = np.linalg.norm(np.array(encodeListKnown) - encodeFace, axis=1)
        matchIndex = np.argmin(distances)
        if distances[matchIndex] < 0.6:  # Threshold for a valid recognition
            correct_recognitions += 1
            name = classNames[matchIndex].upper()
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)

            # Calculate accuracy and display it with the name
            accuracy = (correct_recognitions / total_attempts) * 100 if total_attempts > 0 else 0
            cv2.putText(img, f"{name} {accuracy:.2f}%", (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            markAttendance(name)
            print("Attendance marked for", name)

    # Show live camera image
    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Calculate and print final accuracy
if total_attempts > 0:
    accuracy = (correct_recognitions / total_attempts) * 100
    print(f'Final Accuracy: {accuracy:.2f}%')
else:
    print('No face recognition attempts made.')


In [ ]:
#VGGNet.py

import cv2
import numpy as np
import os
from datetime import datetime
from deepface import DeepFace

# Path to your images folder
path = 'imagesatted'
images = []
classNames = []

# List the files in the images folder
mylist = os.listdir(path)
print(mylist)

# Load images and extract class names
for cls in mylist:
    curImg = cv2.imread(os.path.join(path, cls))
    if curImg is not None:
        images.append(curImg)
        classNames.append(os.path.splitext(cls)[0])
print(classNames)

# Function to find face encodings using DeepFace
def findEncodings(images):
    encodeList = []
    for img in images:
        try:
            encodings = DeepFace.represent(img, model_name='VGG-Face')
            if encodings:
                encodeList.append(encodings[0]["embedding"])
        except Exception as e:
            print(f"Error encoding image: {e}")
    return encodeList

# Function to mark attendance
def markAttendance(name):
    with open('Attendance.csv', 'r+') as f:
        myDataList = f.readlines()
        nameList = [line.split(',')[0] for line in myDataList]
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')

# Find encodings for known faces
encodeListKnown = findEncodings(images)
print('Encoding complete')

# Initialize webcam
cap = cv2.VideoCapture(0)

total_attempts = 0
correct_recognitions = 0

while True:
    success, img = cap.read()
    if not success:
        break
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    try:
        # Detect faces in the current frame
        facesCurFrame = DeepFace.extract_faces(imgS, detector_backend='mtcnn', enforce_detection=False)
    except Exception as e:
        print(f"Detection error: {e}")
        continue

    for face in facesCurFrame:
        faceLoc = face["facial_area"]
        faceImg = face["face"]
        try:
            encodeFace = DeepFace.represent(faceImg, model_name='VGG-Face', enforce_detection=False)[0]["embedding"]
        except Exception as e:
            print(f"Representation error: {e}")
            continue

        distances = np.linalg.norm(np.array(encodeListKnown) - encodeFace, axis=1)
        matchIndex = np.argmin(distances)
        total_attempts += 1  # Increment total attempts
        if distances[matchIndex] < 0.6:  # Threshold for a valid recognition
            correct_recognitions += 1  # Increment correct recognitions
            name = classNames[matchIndex].upper()
            y1, x2, y2, x1 = faceLoc["y"], faceLoc["x"] + faceLoc["w"], faceLoc["y"] + faceLoc["h"], faceLoc["x"]
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)

            # Calculate accuracy and display it with the name
            accuracy = (correct_recognitions / total_attempts) * 100 if total_attempts > 0 else 0
            cv2.putText(img, f"{name} {accuracy:.2f}%", (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            markAttendance(name)
            print("Attendance marked for", name)
        else:
            print(f"No match found. Distances: {distances}")

    # Show live camera image
    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Calculate and print accuracy
if total_attempts > 0:
    accuracy = (correct_recognitions / total_attempts) * 100
    print(f'Accuracy: {accuracy:.2f}%')
else:
    print('No face recognition attempts made.')


In [ ]:
Run gui2.py file and done you can recognize image.